# NDVI calculation with xcube-stac

In [1]:
import xarray as xr
from xcube.core.store import new_data_store

Set credentials, if needed.

In [2]:
credentials = {
    "key": "insert S3 key here",
    "secret": "insert S3 secret here",
}

Instantiate a CDSE STAC store. This will require valid CDSE S3 credentials.

In [3]:
store = new_data_store("stac-cdse", stack_mode=False, **credentials)

Alternative syntax for non-CDSE stores, specifying STAC root URL explicitly.

In [4]:
# store = new_data_store("stac", stack_mode=False, url="https://earth-search.aws.element84.com/v1")

Select a product on which to perform the calculation.

In [5]:
product_id = "S2A_MSIL2A_20250321T102731_N0511_R108_T32UNE_20250321T122920"

Open the dataset and show a summary of its metadata.

In [6]:
ds = store.open_data(
    data_id=f"collections/sentinel-2-l2a/items/{product_id}"
)
ds

<xarray.Dataset> Size: 3GB
Dimensions:      (y: 10980, x: 10980)
Coordinates:
    spatial_ref  int64 8B 0
  * x            (x) float64 88kB 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05
  * y            (y) float64 88kB 6e+06 6e+06 6e+06 ... 5.89e+06 5.89e+06
Data variables: (12/15)
    B01          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    B09          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    B02          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    B03          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    B04          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    B08          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    ...           ...
    B06          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    B07          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    B8A          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    B11          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    B12          (y, x) uint16 241MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
    SCL          (y, x) uint8 121MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>
Attributes:
    stac_catalog_url:  https://stac.dataspace.copernicus.eu/v1
    stac_item_id:      S2A_MSIL2A_20250321T102731_N0511_R108_T32UNE_20250321T...

Calculate the NDVI, store it in a new dataset, and show a summary of the new dataset’s metadata.

In [7]:
ndvi = xr.Dataset(data_vars={"ndvi": (ds.B08 - ds.B04) / (ds.B08 + ds.B04)})
ndvi

<xarray.Dataset> Size: 965MB
Dimensions:      (x: 10980, y: 10980)
Coordinates:
    spatial_ref  int64 8B 0
  * x            (x) float64 88kB 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05
  * y            (y) float64 88kB 6e+06 6e+06 6e+06 ... 5.89e+06 5.89e+06
Data variables:
    ndvi         (y, x) float64 964MB dask.array<chunksize=(1024, 1024), meta=np.ndarray>

Write the data to a local Zarr archive.

In [8]:
ndvi.to_zarr("ndvi.zarr")

/home/pont/mambaforge/envs/xcube-stac/lib/python3.13/site-packages/dask/_task_spec.py:764: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
